<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩 (https://school.fun-coding.org/)</a> 에서 본 강의를 포함하는 최적화된 로드맵도 확인하실 수 있습니다</b></div>

### 가벼운 딥러닝 적용3 - MULTI-CLASS CLASSIFICATION
<div class="alert alert-block" style="border: 2px solid #E65100;background-color:#FFF3E0;padding:10px">
<font size="4em" style="color:#BF360C;">지금까지 익힌 이론과 코드만 가지고도, 바로 딥러닝 적용이 가능합니다</font><br>
<font size="4em" style="color:#BF360C;">빠르게 여러 문제를 딥러닝에 적용해보며, PYTORCH 와 딥러닝에 우선 익숙해지도록 합니다</font>
</div>

In [34]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

# Split the data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [35]:
std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train_tensor = torch.from_numpy(std_scaler.transform(X_train)).float()
X_test_tensor = torch.from_numpy(std_scaler.transform(X_test)).float()
y_train_tensor = torch.from_numpy(y_train).long()
y_test_tensor = torch.from_numpy(y_test).long()

print (X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape)

torch.Size([120, 4]) torch.Size([30, 4]) torch.Size([120]) torch.Size([30])


In [36]:
nb_epochs = 1000 # 1000 epoch 실행 예정
minibatch_size = 120 # Mini-batch 사이즈는 256 으로 정하고, 1 epoch 에 10000 개 데이터를 40개의 Mini-batch 로 나누어 40 iteration 실행

In [41]:
class FunModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        self.linear_layers = nn.Sequential (
            nn.Linear(input_dim, 100),
            nn.LeakyReLU(0.1),
            nn.Linear(100, 20),
            nn.LeakyReLU(0.1),
            nn.Linear(20, 5),
            nn.LeakyReLU(0.1),
            nn.Linear(5, output_dim),
            # nn.Softmax(dim=-1) 
            nn.LogSoftmax(dim=-1) # 최종 결과는 (5, 3) 이 되므로, dim=-1 로 label 확률값이 들어 있는 마지막 차원을 지정해줘야 함
        )
        
    def forward(self, x):
        y = self.linear_layers(x)
        return y

In [42]:
input_dim = X_train_tensor.size(-1) 
output_dim = 3 # iris 는 0, 1, 2 Multi-Label 에 대한 확률값을 구해야 하므로, output dimension 은 3 이 되어야 함
print (input_dim, output_dim)
model = FunModel(input_dim, output_dim)   

# loss_func = nn.CrossEntropyLoss() # softmax 는 CrossEntropyLoss() 로 진행해야 함
loss_func = nn.NLLLoss() # log softmax 는 NLLLoss() 로 진행해야 함
optimizer = torch.optim.Adam(model.parameters()) # Adam, learning rate 필요없음

4 3


In [43]:
for index in range(nb_epochs):
    indices = torch.randperm(X_train_tensor.size(0))

    x_batch_list = torch.index_select(X_train_tensor, 0, index=indices)
    y_batch_list = torch.index_select(y_train_tensor, 0, index=indices)
    x_batch_list = x_batch_list.split(minibatch_size, 0)
    y_batch_list = y_batch_list.split(minibatch_size, 0)

    epoch_loss = list()        
    for x_minibatch, y_minibatch in zip(x_batch_list, y_batch_list):
        y_minibatch_pred = model(x_minibatch)
        loss = loss_func(y_minibatch_pred, y_minibatch)
        epoch_loss.append(loss)        

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (index % 100) == 0:
        print (index, sum(epoch_loss) / len(epoch_loss))        


0 tensor(1.1004, grad_fn=<DivBackward0>)
100 tensor(0.3363, grad_fn=<DivBackward0>)
200 tensor(0.0644, grad_fn=<DivBackward0>)
300 tensor(0.0417, grad_fn=<DivBackward0>)
400 tensor(0.0360, grad_fn=<DivBackward0>)
500 tensor(0.0321, grad_fn=<DivBackward0>)
600 tensor(0.0274, grad_fn=<DivBackward0>)
700 tensor(0.0210, grad_fn=<DivBackward0>)
800 tensor(0.0136, grad_fn=<DivBackward0>)
900 tensor(0.0078, grad_fn=<DivBackward0>)


### 테스트셋 기반 Evaluation

In [20]:
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor)
    # torch.argmax(x, dim=차원) # 특정 차원의 데이터 중 가장 높은 값을 가진 index 값을 리턴
    y_pred_list = torch.argmax(y_test_pred, dim=1) # 3개의 레이블중 값이 가장 큰 값 (확률) 이 해당 레이블 예측값임 (해당 index 를 최종 결과값으로 활용 가능) 

### mini-batch size 기반 예측

In [55]:
y_pred_list = list()
x_test_batch_list = X_test_tensor.split(minibatch_size, 0)
model.eval()
with torch.no_grad():
    for x_minibatch in x_test_batch_list:
        y_test_pred = model(x_minibatch)
        y_test_pred = torch.argmax(y_test_pred, dim=1)
        y_pred_list.extend(y_test_pred.detach().tolist())

y_pred_list = torch.tensor(y_pred_list)

In [56]:
print (y_pred_list.shape, y_test_tensor.shape)

torch.Size([30]) torch.Size([30])


### Multi-Label Classification 기본 매트릭
- None : 라벨 별로 각 계산값 그대로 출력함
- micro : 전체 라벨 값을 합하여 계산함
- macro : 라벨 별로 계산된 값에 대한 전체 평균을 출력함

In [58]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

print("Confusion Matrix\n", str(confusion_matrix(y_test_tensor, y_pred_list)))
print("Precision List:\t", str( precision_score(y_test_tensor, y_pred_list, average=None) ) )
print("Macro Precision:\t", str( precision_score(y_test_tensor, y_pred_list, average='macro' ) ) )
print ("Macro Precision Formula:", str( sum(precision_score(y_test_tensor, y_pred_list, average=None) ) / 3 ))
print("Micro Precision:\t", str( precision_score(y_test_tensor, y_pred_list, average='micro') ) )

print("Recall List:\t", str( precision_score(y_test_tensor, y_pred_list, average=None) ) )
print("Macro Recall:\t", str( recall_score(y_test_tensor, y_pred_list, average='macro') ) )
print("Micro Recall:\t", str( recall_score(y_test_tensor, y_pred_list, average='micro') ) )

print("Macro F1 Score List:\t", str( f1_score(y_test_tensor, y_pred_list, average=None) ) )
print("Macro F1 Score:\t", str( f1_score(y_test_tensor, y_pred_list, average='macro') ) )
print("Micro F1 Score:\t", str( f1_score(y_test_tensor, y_pred_list, average='micro') ) )

Confusion Matrix
 [[10  0  0]
 [ 0 10  0]
 [ 0  0 10]]
Precision List:	 [1. 1. 1.]
Macro Precision:	 1.0
Macro Precision Formula: 1.0
Micro Precision:	 1.0
Recall List:	 [1. 1. 1.]
Macro Recall:	 1.0
Micro Recall:	 1.0
Macro F1 Score List:	 [1. 1. 1.]
Macro F1 Score:	 1.0
Micro F1 Score:	 1.0


<div class="alert alert-block" style="border: 2px solid #1976D2;background-color:#E3F2FD;padding:5px;font-size:0.9em;">
본 자료는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 자료를 외부에 공개하지 말아주세요.<br>
<b><a href="https://school.fun-coding.org/">잔재미코딩 (https://school.fun-coding.org/)</a> 에서 본 강의를 포함하는 최적화된 로드맵도 확인하실 수 있습니다</b></div>